In [1]:
from pathlib import Path
import json

In [2]:
folder = Path("")
dataset_file = folder / "tasks_results.json"

with open(dataset_file, 'r') as file:
    task_results = json.load(file)

# convert all tasks and model names to lower
task_results = {key.lower(): {key_in.lower(): value_in for key_in, value_in in value.items()} for key, value in task_results.items()}

In [3]:
def get_models_set(task_dict):
    # get set of models for each task.
    model_dict = {}
    for task, model_scores in task_dict.items():
        model_set = {model.lower() for model in model_scores.keys()}
        model_dict[task] = model_set

    return

def model_task_count(tasks_dict):
    # Calculate number of tasks, in which models are used
    all_tasks = set(tasks_dict.keys())
    model_analysis = {}

    for task, model_scores in tasks_dict.items():
        for model in model_scores.keys():
            model_lower = model.lower()
            if model_lower not in model_analysis:
                model_analysis[model_lower] = {
                    "num_tasks": 0,
                    "tasks": set(),
                    "missed_tasks": set()
                }
            model_analysis[model_lower]["num_tasks"] += 1
            model_analysis[model_lower]["tasks"].add(task)

    for model, details in model_analysis.items():
        details["missed_tasks"] = list(all_tasks - details["tasks"])
        details["tasks"] = list(details["tasks"])

    sorted_model_analysis = dict(sorted(model_analysis.items(), key=lambda item: item[1]["num_tasks"], reverse=True))

    return sorted_model_analysis

In [4]:
model_dict = get_models_set(task_results)
model_count = model_task_count(task_results)
model_list = sorted(list(model_count.keys()))

In [6]:
if True:
    with open("model_count.json", "w") as file:
        json.dump(model_count, file, indent=4)
    with open("model_list.txt", "w") as file:
        file.write("\n".join(model_list))

In [86]:
# task_score_gpt4o = dict()
#
# for task, model_scores in task_results.items():
#     task_score_gpt4o[task] = model_scores["gpt-4o"]

In [7]:
with open("model_list.txt", "w") as file:
    file.write("\n".join(model_list))